# V2log YOLO26-N Weight Plate Training (v3 — 통합본)

> **GitHub + 데스크탑 노트북의 장점만 합침**
> - ✅ Drive ZIP 방식 (API 키 노출 없음, 연결 끊겨도 결과 안전)
> - ✅ 데이터 시각화 (학습 전 품질 확인)
> - ✅ 증강 파라미터 세부 설정 (소량 데이터 대응)
> - ✅ ONNX(권장) + TFLite(백업) 듀얼 내보내기
> - ✅ 테스트 검증 + 클래스별 AP + 예측 시각화

| 항목 | 값 |
|------|---|
| 데이터셋 | 926장 (5개 클래스) |
| 모델 | **YOLO26-N** (nano, NMS-free, CPU 43% 빠름) |
| GPU | T4 (Colab 무료) |
| 예상 시간 | 30분~2시간 |
| 결과물 | `weight_plate.onnx` (Android 메인) + `weight_plate.tflite` (백업) |

---

## 실행 전 체크리스트

1. **런타임 → 런타임 유형 변경 → T4 GPU** 선택
2. Google Drive에 `V2log-CV-Training` 폴더 + 데이터셋 ZIP 확인
3. 셀을 **위에서 아래로 순서대로** 실행 (Shift+Enter)

> ⚠️ **YOLO11 아닙니다! 반드시 YOLO26-N으로 학습합니다!**

---
## 셀 1: Google Drive 연결 (30초)
▶ 실행 → 팝업에서 Google 계정 선택 → 허용

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print('✅ Drive 연결 완료!')

---
## 셀 2: 데이터셋 압축 해제 (1분)
Drive에 업로드한 ZIP을 Colab 로컬에 풀기

In [ ]:
import zipfile, os, glob

drive_path = '/content/drive/MyDrive/V2log-CV-Training/'

if not os.path.exists(drive_path):
    print('❌ V2log-CV-Training 폴더가 없습니다!')
    print('Google Drive에 V2log-CV-Training 폴더를 만들고 ZIP을 업로드하세요.')
else:
    zip_files = glob.glob(os.path.join(drive_path, '*.zip'))

    if not zip_files:
        print(f'❌ {drive_path} 에 ZIP 파일이 없습니다!')
        print('폴더 안의 파일들:')
        for f in os.listdir(drive_path):
            print(f'  {f}')
    else:
        zip_path = max(zip_files, key=os.path.getmtime)
        print(f'📦 ZIP 발견: {os.path.basename(zip_path)}')
        print('압축 해제 중...')

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall('/content/dataset')

        total = 0
        for split in ['train', 'valid', 'test']:
            img_dir = f'/content/dataset/{split}/images'
            if os.path.exists(img_dir):
                count = len([f for f in os.listdir(img_dir)
                             if f.endswith(('.jpg', '.jpeg', '.png'))])
                total += count
                print(f'  {split}: {count}장')

        print(f'\n✅ 총 {total}장 — 데이터셋 준비 완료!')

---
## 셀 3: YOLO26 설치 + data.yaml 경로 수정 (1분)

ultralytics 최신 버전이어야 YOLO26-N을 사용할 수 있습니다.

In [ ]:
!pip install ultralytics --upgrade -q

import ultralytics, yaml
print(f'ultralytics: {ultralytics.__version__}')

# YOLO26 지원 확인 (v8.4.14 이상 필요)
v = ultralytics.__version__.split('.')
major, minor = int(v[0]), int(v[1])
patch = int(v[2]) if len(v) > 2 else 0
if major >= 8 and (minor > 4 or (minor == 4 and patch >= 14)):
    print('✅ YOLO26-N 지원 확인!')
else:
    print(f'⚠️ v8.4.14 이상 필요! 현재 {ultralytics.__version__}')
    print('!pip install ultralytics --upgrade 를 다시 실행하세요')

# data.yaml 경로를 Colab 환경에 맞게 수정
yaml_path = '/content/dataset/data.yaml'
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

data['path'] = '/content/dataset'
data['train'] = 'train/images'
data['val'] = 'valid/images'
data['test'] = 'test/images'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print(f'\n클래스 수: {data["nc"]}')
print(f'클래스: {data["names"]}')
print('\n✅ data.yaml 경로 수정 완료!')

---
## 셀 4: 데이터 시각화 — 바운딩 박스 + 클래스 분포 확인

학습 전에 **이미지와 라벨이 정상인지** 시각적으로 확인합니다.
문제가 보이면 Roboflow에서 수정 후 ZIP 재생성.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random, os, yaml

with open('/content/dataset/data.yaml') as f:
    data_config = yaml.safe_load(f)
CLASS_NAMES = data_config['names']

COLORS = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF',
          '#00FFFF', '#FFA500', '#800080', '#008000']

train_img_dir = '/content/dataset/train/images'
train_lbl_dir = '/content/dataset/train/labels'

all_images = [f for f in os.listdir(train_img_dir)
              if f.endswith(('.jpg', '.jpeg', '.png'))]
samples = random.sample(all_images, min(8, len(all_images)))

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, img_name in enumerate(samples):
    img = Image.open(os.path.join(train_img_dir, img_name))
    w, h = img.size
    axes[i].imshow(img)

    lbl_path = os.path.join(train_lbl_dir,
                            os.path.splitext(img_name)[0] + '.txt')
    if os.path.exists(lbl_path):
        with open(lbl_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls_id = int(parts[0])
                    xc, yc, bw, bh = [float(x) for x in parts[1:5]]
                    x1 = (xc - bw / 2) * w
                    y1 = (yc - bh / 2) * h
                    color = COLORS[cls_id % len(COLORS)]
                    rect = patches.Rectangle(
                        (x1, y1), bw * w, bh * h,
                        linewidth=2, edgecolor=color, facecolor='none')
                    axes[i].add_patch(rect)
                    name = (CLASS_NAMES[cls_id] if isinstance(CLASS_NAMES, list)
                            else CLASS_NAMES.get(cls_id, f'cls_{cls_id}'))
                    axes[i].text(x1, y1 - 5, name, color=color, fontsize=8,
                                bbox=dict(boxstyle='round,pad=0.2',
                                          facecolor='black', alpha=0.7))

    axes[i].set_title(img_name[:25], fontsize=8)
    axes[i].axis('off')

plt.suptitle('학습 데이터 샘플 (바운딩 박스 확인)', fontsize=14)
plt.tight_layout()
plt.show()

# === 클래스별 분포 ===
class_counts = {}
for lbl_file in os.listdir(train_lbl_dir):
    if lbl_file.endswith('.txt'):
        with open(os.path.join(train_lbl_dir, lbl_file)) as f:
            for line in f:
                cls_id = int(line.strip().split()[0])
                name = (CLASS_NAMES[cls_id] if isinstance(CLASS_NAMES, list)
                        else CLASS_NAMES.get(cls_id, f'cls_{cls_id}'))
                class_counts[name] = class_counts.get(name, 0) + 1

print('\n클래스별 어노테이션 분포 (train):')
for name, count in sorted(class_counts.items(), key=lambda x: -x[1]):
    bar = '█' * (count // 3)
    print(f'  {name:<15} {count:>4}개  {bar}')
print(f'\n총 어노테이션: {sum(class_counts.values())}개')

---
## 셀 5: YOLO26-N 학습 시작 (30분~2시간)

**이 셀이 핵심입니다!**

- 진행 상황이 실시간 출력됩니다 (Epoch 1/150, loss, mAP 등)
- **Colab 탭을 닫지 마세요!** (연결 끊기면 학습 중단)
- `patience=25`: 25 에폭 동안 개선 없으면 자동 종료
- `close_mosaic=15`: 마지막 15 에폭은 모자이크 끄고 안정화
- 926장 소량 데이터 → 증강 파라미터를 강하게 설정했습니다

> **반드시 `yolo26n.pt`** — yolo11n.pt 절대 사용 금지!

In [ ]:
from ultralytics import YOLO

# ============================================
#   YOLO26-N (절대 yolo11n 사용 금지!)
# ============================================
model = YOLO('yolo26n.pt')
print(f'모델 로드 완료!')

results = model.train(
    data='/content/dataset/data.yaml',
    epochs=150,             # 최대 150 에폭
    imgsz=640,              # 이미지 크기
    batch=16,               # 배치 크기 (T4: 16 적당)
    device=0,               # GPU 사용
    patience=25,            # 25 에폭 개선 없으면 조기 종료
    save=True,              # 체크포인트 저장
    project='/content/models',
    name='weight_plate_yolo26n',
    exist_ok=True,
    # === 증강 (926장 소량 데이터 → 강하게) ===
    mosaic=1.0,             # 4장 합치기
    close_mosaic=15,        # 마지막 15 에폭은 모자이크 OFF
    mixup=0.15,             # 이미지 섞기 (기본 0.0 → 올림)
    flipud=0.1,             # 상하 반전 (기본 0.0 → 올림)
    fliplr=0.5,             # 좌우 반전
    degrees=15.0,           # 회전 ±15도 (기본 0.0 → 올림)
    translate=0.2,          # 이동 20% (기본 0.1 → 올림)
    scale=0.5,              # 크기 변화
    hsv_h=0.015,            # 색상 변화
    hsv_s=0.7,              # 채도 변화
    hsv_v=0.4,              # 밝기 변화
    erasing=0.4,            # 랜덤 지우기 (가림 대응)
)

# 결과 출력
mAP50 = results.results_dict.get('metrics/mAP50(B)', 0)
mAP5095 = results.results_dict.get('metrics/mAP50-95(B)', 0)

print(f'\n{"=" * 50}')
print(f'YOLO26-N 학습 완료!')
print(f'{"=" * 50}')
print(f'mAP50:    {mAP50:.1%}')
print(f'mAP50-95: {mAP5095:.1%}')

if mAP50 >= 0.8:
    print('\n✅ 80% 이상! 다음 셀로 진행하세요!')
elif mAP50 >= 0.6:
    print('\n⚠️ 60~80% — 나쁘지 않음. 일단 다음 셀 진행.')
else:
    print('\n❌ 60% 미만 — 데이터 추가 필요할 수 있습니다.')

---
## 셀 6: 학습 결과 시각화

학습 곡선(loss, mAP), 혼동 행렬, PR 곡선을 확인합니다.

In [ ]:
from IPython.display import Image as IPImage, display
import os

result_dir = '/content/models/weight_plate_yolo26n'

charts = [
    ('results.png', '학습 곡선 (loss + mAP)', 900),
    ('confusion_matrix_normalized.png', '혼동 행렬 (클래스별 정확도)', 600),
    ('PR_curve.png', 'PR 곡선', 600),
    ('val_batch0_pred.jpg', '검증 데이터 예측 결과', 900),
]

for filename, desc, width in charts:
    path = os.path.join(result_dir, filename)
    if os.path.exists(path):
        print(f'\n📊 {desc}:')
        display(IPImage(filename=path, width=width))
    else:
        print(f'⚠️ {filename} 없음 — 학습이 완료되었는지 확인하세요')

---
## 셀 7: 테스트 데이터 검증 + 예측 시각화

학습에 안 쓴 테스트 데이터로 **실제 성능**을 확인합니다.
클래스별 AP와 예측 결과를 시각적으로 보여줍니다.

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import random, os

# 최고 성능 모델 로드
best_model = YOLO('/content/models/weight_plate_yolo26n/weights/best.pt')

# 테스트 세트 검증
val_results = best_model.val(
    data='/content/dataset/data.yaml',
    split='test',
    imgsz=640,
    batch=16,
)

test_mAP50 = val_results.results_dict.get('metrics/mAP50(B)', 0)
test_mAP5095 = val_results.results_dict.get('metrics/mAP50-95(B)', 0)
precision = val_results.results_dict.get('metrics/precision(B)', 0)
recall = val_results.results_dict.get('metrics/recall(B)', 0)

print(f'{"=" * 50}')
print(f'테스트 데이터 검증 결과')
print(f'{"=" * 50}')
print(f'mAP50:     {test_mAP50:.1%}')
print(f'mAP50-95:  {test_mAP5095:.1%}')
print(f'Precision: {precision:.1%}')
print(f'Recall:    {recall:.1%}')

# 클래스별 AP
print(f'\n클래스별 AP50:')
if hasattr(val_results, 'box') and hasattr(val_results.box, 'ap50'):
    names = val_results.names
    for i in range(len(val_results.box.ap50)):
        ap = val_results.box.ap50[i]
        cls_name = names.get(i, f'cls_{i}') if isinstance(names, dict) else names[i]
        status = '✅' if ap >= 0.8 else '⚠️' if ap >= 0.6 else '❌'
        print(f'  {status} {cls_name:<15} {ap:.1%}')

# 테스트 이미지 예측 시각화
test_img_dir = '/content/dataset/test/images'
if os.path.exists(test_img_dir):
    test_images = [f for f in os.listdir(test_img_dir)
                   if f.endswith(('.jpg', '.jpeg', '.png'))]
    if test_images:
        samples = random.sample(test_images, min(8, len(test_images)))
        fig, axes = plt.subplots(2, 4, figsize=(20, 10))
        axes = axes.flatten()

        for i, img_name in enumerate(samples):
            img_path = os.path.join(test_img_dir, img_name)
            pred = best_model.predict(img_path, imgsz=640, conf=0.25, verbose=False)[0]
            result_img = pred.plot()[:, :, ::-1]  # BGR → RGB
            axes[i].imshow(result_img)
            axes[i].set_title(f'{img_name[:20]} ({len(pred.boxes)}개)', fontsize=9)
            axes[i].axis('off')

        plt.suptitle('테스트 이미지 예측 결과', fontsize=14)
        plt.tight_layout()
        plt.show()

print(f'\n{"=" * 50}')
if test_mAP50 >= 0.8:
    print('✅ 합격! 다음 셀에서 모델을 내보내세요!')
elif test_mAP50 >= 0.6:
    print('⚠️ 준수하지만 개선 여지 있음')
else:
    print('❌ 데이터 추가 후 재학습 권장')

---
## 셀 8: 모델 내보내기 + Google Drive 저장

**YOLO26은 TFLite에서 Android GPU delegate 호환 문제가 있습니다.**
그래서 **ONNX를 메인**으로, TFLite는 백업용으로 내보냅니다.

| 형식 | Android 호환 | 역할 |
|------|:-----------:|------|
| **ONNX** | ✅ NNAPI 가속 | **메인** |
| TFLite | ⚠️ GPU delegate 이슈 | 백업 |

결과물은 **Google Drive에 자동 저장**됩니다 (연결 끊겨도 안전!)

In [ ]:
from ultralytics import YOLO
import shutil, os

best_pt = '/content/models/weight_plate_yolo26n/weights/best.pt'
model = YOLO(best_pt)

# ============================================
# 1) ONNX 내보내기 (Android 권장)
# ============================================
print('ONNX 변환 중...')
onnx_path = model.export(
    format='onnx',
    imgsz=640,
    simplify=True,    # 그래프 최적화
    opset=17,          # ONNX opset 17
    half=True,         # FP16
)
onnx_size = os.path.getsize(onnx_path) / 1024 / 1024
print(f'✅ ONNX 완료: {onnx_size:.1f} MB')

# ============================================
# 2) TFLite 내보내기 (백업용)
# ============================================
print('\nTFLite 변환 중... (1~3분)')
tflite_path = None
try:
    model_tflite = YOLO(best_pt)
    tflite_path = model_tflite.export(
        format='tflite',
        imgsz=640,
        half=False,    # FP32 (호환성 우선)
        int8=False,
    )
    tflite_size = os.path.getsize(tflite_path) / 1024 / 1024
    print(f'✅ TFLite 완료: {tflite_size:.1f} MB')
except Exception as e:
    print(f'⚠️ TFLite 변환 실패 (YOLO26 알려진 이슈): {e}')
    print('   → ONNX 파일을 사용하세요 (Android NNAPI 지원)')

# ============================================
# 3) Google Drive에 결과물 저장
# ============================================
dst = '/content/drive/MyDrive/V2log-CV-Training/results_yolo26n/'
os.makedirs(dst, exist_ok=True)

# 모델 가중치 파일
weights_dir = '/content/models/weight_plate_yolo26n/weights/'
for f in os.listdir(weights_dir):
    src = os.path.join(weights_dir, f)
    if os.path.isfile(src):
        shutil.copy2(src, dst)
        size_mb = os.path.getsize(src) / (1024 * 1024)
        print(f'  저장: {f} ({size_mb:.1f} MB)')

# ONNX 파일 (이름 통일)
if os.path.exists(onnx_path):
    shutil.copy2(onnx_path, os.path.join(dst, 'weight_plate.onnx'))
    print(f'  저장: weight_plate.onnx')

# TFLite 파일 (이름 통일)
if tflite_path and os.path.exists(tflite_path):
    shutil.copy2(tflite_path, os.path.join(dst, 'weight_plate.tflite'))
    print(f'  저장: weight_plate.tflite')

# 학습 결과 그래프/CSV
result_base = '/content/models/weight_plate_yolo26n/'
for f in os.listdir(result_base):
    src = os.path.join(result_base, f)
    if os.path.isfile(src) and (f.endswith('.png') or f.endswith('.csv')):
        shutil.copy2(src, dst)

print(f'\n{"=" * 50}')
print(f'✅ 모든 결과물 Google Drive에 저장 완료!')
print(f'{"=" * 50}')
print(f'위치: Drive > V2log-CV-Training > results_yolo26n/')
print(f'\n📦 weight_plate.onnx  → 앱 assets/models/ (Android 메인)')
if tflite_path:
    print(f'📦 weight_plate.tflite → assets/models/ (백업)')
print(f'📦 best.pt            → 재학습용 보관')
print(f'\n🎉 Phase 2A 완료! → Phase 2B (앱 통합) 시작!')

---
## (선택) 셀 9: 추가 학습 — mAP50 < 80%일 때만

기존 결과에서 이어서 학습합니다. **주석(#)을 제거하고 실행하세요.**

In [ ]:
# === 추가 학습 (성능 부족 시에만 주석 제거 후 실행) ===

# from ultralytics import YOLO
#
# model = YOLO('/content/models/weight_plate_yolo26n/weights/best.pt')
#
# results = model.train(
#     data='/content/dataset/data.yaml',
#     epochs=50,              # 추가 50 에폭
#     imgsz=640,
#     batch=16,
#     device=0,
#     patience=15,
#     save=True,
#     project='/content/models',
#     name='weight_plate_yolo26n_ft',
#     lr0=0.001,              # 학습률 낮춰서 미세 조정
#     close_mosaic=10,
# )
#
# print(f"mAP50: {results.results_dict.get('metrics/mAP50(B)', 0):.1%}")

print('이 셀은 주석 처리되어 있습니다.')
print('mAP50 < 80%일 때만 주석(#)을 제거하고 실행하세요.')

---
## 문제 해결

| 문제 | 해결 |
|------|------|
| GPU가 없다고 나옴 | 런타임 → 런타임 유형 변경 → T4 GPU |
| Colab 연결 끊김 | 다시 연결 후 **셀 5부터** (체크포인트에서 이어감) |
| mAP가 매우 낮음 (<40%) | 라벨링 품질 점검 (Roboflow에서 확인) |
| 특정 클래스만 낮음 | 해당 무게 사진 200~300장 추가 후 재학습 |
| TFLite 변환 실패 | YOLO26 알려진 이슈 — ONNX를 메인으로 사용 |
| ONNX 변환 실패 | `!pip install -U ultralytics onnx` 후 재시도 |
| CUDA 메모리 부족 | 셀 5에서 `batch=8`로 줄이기 |
| ZIP 파일 못 찾음 | Drive > V2log-CV-Training 폴더에 ZIP 있는지 확인 |

---

## 전체 흐름 요약

```
셀 1:  Google Drive 연결 (30초)
셀 2:  데이터셋 압축 해제 (1분)
셀 3:  YOLO26 설치 + data.yaml 수정 (1분)
셀 4:  데이터 시각화 — 품질 확인 ★
셀 5:  ⭐ YOLO26-N 학습 (30분~2시간)
셀 6:  학습 결과 시각화 ★
셀 7:  테스트 검증 + 클래스별 AP ★
셀 8:  ONNX + TFLite 내보내기 + Drive 저장
셀 9:  (선택) 추가 학습
```

★ = v3에서 새로 추가된 셀

---

**검증 출처:**
- [Ultralytics Train Docs](https://docs.ultralytics.com/modes/train/)
- [Ultralytics Export Docs](https://docs.ultralytics.com/modes/export/)
- [YOLO26 TFLite Issue #23282](https://github.com/ultralytics/ultralytics/issues/23282)
- [PyTorch CUDA API](https://pytorch.org/docs/stable/generated/torch.cuda.get_device_properties.html)